# Задача 4.2

Рассмотрим некоторую задачу классификации. Пусть задано качество 4 моделей $a_1, a_2, a_3, a_4$. Качество полученных моделей показано в [таблице](data/classifiers.csv). 

Исследователю требуется выбрать наилучшую модель. Для выбора лучшей модели требуется попарно сравнить среднее значение качества всех моделей. Может ли исследователь утверждать что какая-то из моделей лучше другой?

**Требуется:**
1. Записать задачу формально.
2. Предложить статистику для решения данной задачи.
3. Записать нулевое распределение данной статистики.
4. Записать явно правило принятия решения на основе статистики и нулевого распределения для обеспечения уровня значимости $\alpha = 0.05$.
5. Проверить гипотезу по записанному критерию для данных из условия. Противоречат ли они гипотезе?

Все выкладки должны быть сделаны аналитически, без использования компьютера. Допускается использование компютера для подстановки численых значений в финальную формулу.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from tqdm import tqdm_notebook as tqdm

import seaborn as sns

import matplotlib.pyplot as plt

from permute.core import two_sample

from arch.bootstrap import IIDBootstrap

from statsmodels.stats.diagnostic import kstest_normal

from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score

import itertools

from statsmodels.stats.multitest import multipletests

In [2]:
def get_scores(model, X_train, y_train, X_test, y_test, metric = roc_auc_score):
    
    a = np.unique(y_train)
    pair = list(itertools.combinations(range(a.shape[0]), 2))
    
    List_of_auc = []
    List_of_acc = []
    
    for i, j in pair:
        indexes = np.where((y_train == i) + (y_train == j))[0]
        X_train_new = X_train[indexes]
        y_train_new = np.array(y_train[indexes])
        indexes = np.where((y_test == i) + (y_test == j))[0]
        X_test_new = X_test[indexes]
        y_test_new = np.array(y_test[indexes])
        
        y_train_new[np.where(y_train_new == i)] = np.zeros_like(y_train_new[np.where(y_train_new == i)])
        y_train_new[np.where(y_train_new == j)] = np.ones_like(y_train_new[np.where(y_train_new == j)])
        y_test_new[np.where(y_test_new == i)] = np.zeros_like(y_test_new[np.where(y_test_new == i)])
        y_test_new[np.where(y_test_new == j)] = np.ones_like(y_test_new[np.where(y_test_new == j)])
        
        model.fit(X_train_new, y_train_new)
        
        y_proba = model.predict_proba(X_test_new)
        y_redict = model.predict(X_test_new)
        
        List_of_auc.append(roc_auc_score(y_test_new, y_proba[:, 1]))
        List_of_acc.append((y_redict == y_test_new).sum()/y_test_new.shape[0])
    
    return List_of_auc, List_of_acc

In [3]:
data = pd.read_csv("data/classifiers.csv")
data.sampleple(5)

,Номер выборки,a1,a2,a3,a4
5,6,5,68,35,17
1,2,85,74,55,35
3,4,44,41,56,37
0,1,86,50,93,13
2,3,53,92,58,51
